In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl

In [2]:
dfRows = pl.read_parquet("./saved_bolillas.parquet")

display(dfRows.head(10))

id,combo,b1,b2,b3,b4,b5,b6,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_left_b6,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,score_r8_b6,shared_values,shared_values_count,score_row
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,bool,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64
"""010203040507""","[1, 2, … 7]",1,2,3,4,5,7,true,"""REGULAR_JUMPS""",1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.5,0.0222,"[0.0222, 1.0]",2,5.6332
"""010203040508""","[1, 2, … 8]",1,2,3,4,5,8,true,"""REGULAR_JUMPS""",1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.3333,0.0222,"[0.0222, 1.0]",2,5.4665
"""010203040509""","[1, 2, … 9]",1,2,3,4,5,9,true,"""REGULAR_JUMPS""",1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.25,0.0222,"[0.0222, 1.0]",2,5.3832
"""010203040510""","[1, 2, … 10]",1,2,3,4,5,10,true,"""REGULAR_JUMPS""",1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.2,0.0222,"[0.0222, 1.0]",2,5.3332
"""010203040511""","[1, 2, … 11]",1,2,3,4,5,11,true,"""REGULAR_JUMPS""",1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.1667,0.0222,"[0.0222, 1.0]",2,5.2999
"""010203040512""","[1, 2, … 12]",1,2,3,4,5,12,true,"""REGULAR_JUMPS""",1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.1429,0.0222,"[0.0222, 1.0]",2,5.2761
"""010203040513""","[1, 2, … 13]",1,2,3,4,5,13,true,"""REGULAR_JUMPS""",1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.125,0.0222,"[0.0222, 1.0]",2,5.2582
"""010203040514""","[1, 2, … 14]",1,2,3,4,5,14,true,"""REGULAR_JUMPS""",1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.1111,0.0222,"[0.0222, 1.0]",2,5.2443
"""010203040515""","[1, 2, … 15]",1,2,3,4,5,15,true,"""REGULAR_JUMPS""",1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.1,0.0222,"[0.0222, 1.0]",2,5.2332


# CHECK IF COULD WON

In [ ]:
ticket = "13 42 41 44 36 20"
yapa = "14"
adicionales = "02 01 18"

def generate_id (int_ticket: list[int]):
  return "".join([str.zfill(str(t), 2)  for t in int_ticket])

def generate_yapas (int_ticket: list[int], yapa: int):
  combos_5 = list(combinations(int_ticket, 5))
  result = []
  for i in range(len(combos_5)):
    row = [*combos_5[i], yapa]
    row.sort()
    result.append(row)

  return result

def generate_adicionales (int_ticket: list[int], adicionales: list[int]):
  combos_5 = list(combinations(int_ticket, 5))
  result = []
  for i in range(len(combos_5)):
    for j in range(len(adicionales)):
      row = [*combos_5[i], adicionales[j]]
      row.sort()
      result.append(row)

  return result

def is_winner_ticket (df: pl.DataFrame, tickets_id: list[str]):
  a = df.filter(
    pl.col("id").is_in(tickets_id)
  )

  is_winner = len(a) > 0
  if (is_winner):
    print(a)

  return len(a) > 0

def get_tickets_ids (ticket: str, yapa: str, adicionales: str):
  int_ticket = [int(t) for t in ticket.split(" ")]
  int_yapa = int(yapa)
  int_adicionales = [int(a) for a in adicionales.split(" ")]

  int_ticket.sort()

  id_ticket = generate_id(int_ticket)
  yapas = generate_yapas(int_ticket=int_ticket, yapa=int_yapa)
  adicionales = generate_adicionales(int_ticket=int_ticket, adicionales=int_adicionales)

  return id_ticket, [generate_id(y) for y in yapas], [generate_id(a) for a in adicionales]

def check_could_won (df: pl.DataFrame, ticket: str, yapa: str, adicionales: str):
  ticket_id, yapas, adicionales = get_tickets_ids(ticket=ticket, yapa=yapa, adicionales=adicionales)


  if (is_winner_ticket(df=df, tickets_id=[ticket_id])):
    print("Holy sht! YOU WON!!!!")

  if (is_winner_ticket(df=df, tickets_id=yapas)):
    print("You won with yapa!!!!")

  if (is_winner_ticket(df=df, tickets_id=adicionales)):
    print("You won with adicionales!!!!")

check_could_won(df=dfRows, ticket=ticket, yapa=yapa, adicionales=adicionales)

shape: (1, 25)
┌──────────────┬─────────────┬─────┬─────┬───┬─────────────┬─────────────┬─────────────┬───────────┐
│ id           ┆ combo       ┆ b1  ┆ b2  ┆ … ┆ score_r8_b6 ┆ shared_valu ┆ shared_valu ┆ score_row │
│ ---          ┆ ---         ┆ --- ┆ --- ┆   ┆ ---         ┆ es          ┆ es_count    ┆ ---       │
│ str          ┆ array[u8,   ┆ u8  ┆ u8  ┆   ┆ f64         ┆ ---         ┆ ---         ┆ f64       │
│              ┆ 6]          ┆     ┆     ┆   ┆             ┆ list[f64]   ┆ u8          ┆           │
╞══════════════╪═════════════╪═════╪═════╪═══╪═════════════╪═════════════╪═════════════╪═══════════╡
│ 132036414244 ┆ [13, 20, …  ┆ 13  ┆ 20  ┆ … ┆ 0.125       ┆ [0.125]     ┆ 1           ┆ 0.7384    │
│              ┆ 44]         ┆     ┆     ┆   ┆             ┆             ┆             ┆           │
└──────────────┴─────────────┴─────┴─────┴───┴─────────────┴─────────────┴─────────────┴───────────┘
Holy sht! YOU WON!!!!
shape: (6, 25)
┌──────────────┬─────────────┬─────┬───

# ANALYZE

In [3]:
dfRows.group_by(
  'reason'
).agg(pl.count('reason').alias('count'))

reason,count
str,u32
"""REGULAR_JUMPS""",3618184
"""""",11345692


In [4]:
def show_b_stats (col: str):
  df_viz = dfRows.group_by(
    col
  ).agg(pl.count(col).alias('count'))

  chart = df_viz.plot.bar(
    x=col,
    y='count',
    color=col,
  )

  display(df_viz)
  display(chart)


In [5]:
show_b_stats('score_left_b1')
show_b_stats('score_r8_b1')


score_left_b1,count
f64,u32
0.125,777094
0.0476,418800
0.025,4924
0.0233,638
0.0588,592735
…,…
0.0227,217
0.0909,785974
0.0526,506622


alt.Chart(...)

score_r8_b1,count
f64,u32
0.125,777094
0.0476,418800
0.025,4924
0.0233,638
0.0588,592735
…,…
0.0385,226450
0.0909,785974
0.0526,506622


alt.Chart(...)

In [6]:
show_b_stats('score_left_b2')
show_b_stats('score_r8_b2')


score_left_b2,count
f64,u32
0.025,1058755
0.125,790
0.0233,1374457
0.0476,52752
0.0588,19664
…,…
0.0385,141144
0.0909,2993
0.0556,26168


alt.Chart(...)

score_r8_b2,count
f64,u32
0.025,45564
0.0476,592631
0.125,341834
0.0233,9445
0.0769,571335
…,…
0.0385,527682
0.0909,440063
0.0556,669290


alt.Chart(...)